In [58]:
import random
import string
import math

p = 900000058987

def pick_random(prime):
    k = random.randint(2, prime - 1)
    return k 

def compute_powers_mod_p(k, p, n):
    powers = [1 for i in range(n)]
    for i in range(1, n):
        powers[i] = (powers[i - 1] * k) % p
    return powers




In [74]:



class StringHasher:
    def __init__(self, s, p, k, powers_mod_p):
        self.s = s
        self.p = p
        self.k = k
        self.powers_mod_p = powers_mod_p
        self.A = self.replace_letters_with_numbers(self.s)
        self.prefix_hashes = self.compute_prefix_hashes(self.A, self.p, self.k)

    
    def replace_letters_with_numbers(self, s):
        letters = list(string.ascii_uppercase)
        letters.append('z')
        letter_to_number = {letters[i] : i + 1 for i in range(len(letters))}
        replaced = [letter_to_number[c] for c in s]
        return replaced
    
    def compute_prefix_hashes(self, A, p, k):
        n = len(A)
        prefix_hashes = [0 for i in range(n)]
        prefix_hashes[0] = A[0]
        for i in range(1, n):
            prefix_hashes[i] = (self.k * prefix_hashes[i - 1] + A[i]) % self.p
        return prefix_hashes

    def compute_hash(self, i, j):
        if i == 0:
            return self.prefix_hashes[j] 
        return (self.prefix_hashes[j] - self.powers_mod_p[j - i + 1] * self.prefix_hashes[i - 1]) % self.p
        

def find_longest_shared_substring(index_and_hasher1, index_and_hasher2):
    i1, hasher1 = index_and_hasher1
    i2, hasher2 = index_and_hasher2
    
    n1 = len(hasher1.s)
    n2 = len(hasher2.s)

    longest_possible = min(n1 - i1, n2 - i2)
    # -1 because a step of 0 is a match of length 1, so step length = match length - 1
    max_step_size = longest_possible - 1
    min_step_size = 0
    curr_step_size = math.ceil((max_step_size + min_step_size) / 2)
    while True:
        if max_step_size == min_step_size:
            if hasher1.A[i1 + curr_step_size] == hasher2.A[i2 + curr_step_size]:
                # We add 1 because the function returns the length of the string
                return (curr_step_size + 1)
            else:
                return curr_step_size
        if hasher1.compute_hash(i1, i1 + curr_step_size) == hasher2.compute_hash(i2, i2 + curr_step_size):
            min_step_size = curr_step_size
        else:
            max_step_size = curr_step_size - 1
        curr_step_size = math.ceil((min_step_size + max_step_size) / 2)



In [72]:
def morganAndString(a, b):
    # We add 'z' to the end of both strings. In python, 'z' > 'A', ..., 'Z' for all uppercase letters.
    # So this is adding a terminal character that is larger than all other characters. Useful for edge cases.
    a = a + 'z'
    b = b + 'z'
    n_a = len(a)
    n_b = len(b)
    i_a = i_b = 0
    p = 900000058987
    k = pick_random(p)
    powers_mod_p = compute_powers_mod_p(p = p, k = k, n = max(n_a, n_b))
    hasher_a = StringHasher(a, p, k, powers_mod_p)
    hasher_b = StringHasher(b, p, k, powers_mod_p)
    output = ""
    shared_substring_length = dict()
    while i_a < n_a and i_b < n_b:
        if a[i_a] < b[i_b]:
            output += a[i_a]
            i_a += 1
        elif a[i_a] > b[i_b]:
            output += b[i_b]
            i_b += 1
        # If the strings are a repeated character 2 characters or more, just alternate
        # which one we pick.
        else:
            if (i_a, i_b) in shared_substring_length:
                L = shared_substring_length[(i_a, i_b)]
            else:
                L = find_longest_shared_substring((i_a, hasher_a), (i_b, hasher_b))
                for k in range(1, L + 1):
                    shared_substring_length[(i_a + k, i_b + k)] = L - k

            if i_a + L == n_a or i_b + L == n_b:
                # In this case, it doesn't matter which we choose.
                output += a[i_a]
                i_a += 1
                continue

            # The longest shared substring has length L. So a[i_a + L]
            # and b[i_b + L] will be the first character at which they disagree.
            if a[i_a + L] < b[i_b + L]:
                output += a[i_a]
                i_a += 1
            else:
                output += b[i_b]
                i_b += 1
                    
    if i_a == n_a:
        for j in range(i_b, n_b):
            output += b[j]
    elif i_b == n_b:
        for j in range(i_a, n_a):
            output += a[j]
    
    # Get rid of the the lowercase 'z's.
    return output[:-2]
        


In [77]:
a = "..."

In [78]:
b = "..."

In [79]:
answer = morganAndString(a, b)

In [63]:
print(len(a))
print(len(b))

65536
65535


In [80]:
expected_answer = "..."

In [81]:
for i in range(len(answer)):
    if answer[i] != expected_answer[i]:
        print(i)
        break

k = 65537

